# prompt

```text
`场地1`和`场地2`构成有向图中的边，但是每条边的数据并不完整，完整数据应该包含从`2021-01-01`到`2022-12-31`。
我需要用该条边已有数据的平均值填充，但是不要覆盖已经存在的值。给出python代码
数据如下，第一行是表头

场地1,场地2,日期,货量
0,DC3,DC5,2021-01-01,3
1,DC3,DC10,2021-01-01,4
2,DC3,DC14,2021-01-01,4
3,DC5,DC3,2021-01-01,41
4,DC5,DC9,2021-01-01,3
5,DC5,DC10,2021-01-01,140
6,DC5,DC14,2021-01-01,57
7,DC5,DC22,2021-01-01,3
8,DC7,DC3,2021-01-01,254
9,DC7,DC5,2021-01-01,203

```

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

plt.rcParams['font.sans-serif'] = ['SimHei']  # 黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决无法显示符号的问题
palette = 'deep'
sns.set(font='SimHei', font_scale=1.2, style='whitegrid', palette=palette)  # 解决Seaborn中文显示问题

rawData = pd.read_excel('../原始数据/附件1：物流网络历史货量数据.xlsx', parse_dates=['日期'], index_col='日期')
print(rawData.dtypes)
rawData

场地1    object
场地2    object
货量      int64
dtype: object


,场地1,场地2,货量
日期,,,
2021-01-01,DC3,DC5,3
2021-01-01,DC3,DC10,4
2021-01-01,DC3,DC14,4
2021-01-01,DC5,DC3,41
2021-01-01,DC5,DC9,3
...,...,...,...
2022-12-31,DC73,DC5,2479
2022-12-31,DC73,DC8,150
2022-12-31,DC73,DC10,8116


In [2]:
from datetime import datetime

date_format = "%Y-%m-%d"
start_date = datetime.strptime("2021-01-01", date_format)
end_date = datetime.strptime("2022-12-31", date_format)

delta = end_date - start_date
deltaDays = delta.days + 1
deltaDays

730

In [3]:
df = rawData.copy()
# 提取出每一条线路
grouped = df.groupby(by=['场地1', '场地2']).size().reset_index(name='count')
print('缺失数据的线路个数', len(grouped[grouped['count'] < deltaDays]))
print('总线路个数', len(grouped))
print('若补全所有线路，则总数据条数', len(grouped) * deltaDays)
grouped

缺失数据的线路个数 1033
总线路个数 1049
若补全所有线路，则总数据条数 765770


,场地1,场地2,count
0,DC1,DC8,1
1,DC10,DC12,211
2,DC10,DC13,21
3,DC10,DC14,730
4,DC10,DC17,9
...,...,...,...
1044,DC9,DC58,1
1045,DC9,DC62,3
1046,DC9,DC67,8
1047,DC9,DC79,1


In [4]:
newDf = pd.DataFrame()
newNoneDf = pd.DataFrame()
# 针对每一条线路判断是否缺失，并补全缺失值
for idx, row in grouped.iterrows():
    print(idx)
    currentDf = df[(df['场地1'] == row['场地1']) & (df['场地2'] == row['场地2'])]
    if row['count'] == deltaDays:
        newDf = pd.concat([newDf, currentDf])
        newNoneDf = pd.concat([newNoneDf, currentDf])
        continue
    # 找出缺失的日期段
    missing_dates = pd.date_range(start=df.index.min(), end=df.index.max()).difference(currentDf.index)

    # 计算平均值
    meanData = currentDf['货量'].mean()
    meanData = round(meanData)
    meanRow = pd.DataFrame({
        '场地1': row['场地1'],
        '场地2': row['场地2'],
        '货量': meanData,
    }, index=[1])
    missingRows = pd.concat([meanRow] * len(missing_dates), ignore_index=True)
    missingRows.index = missing_dates
    missingRows = pd.concat([missingRows, currentDf])
    newDf = pd.concat([newDf, missingRows])

    noneRow = pd.DataFrame({
        '场地1': row['场地1'],
        '场地2': row['场地2'],
        '货量': None,
    }, index=[1])
    missingRows = pd.concat([noneRow] * len(missing_dates), ignore_index=True)
    missingRows.index = missing_dates
    missingRows = pd.concat([missingRows, currentDf])
    newNoneDf = pd.concat([newNoneDf, missingRows])




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
print(len(newDf), len(newNoneDf), )

765770 765770


In [6]:
newDf.head()

,场地1,场地2,货量
2021-01-01,DC1,DC8,3
2021-01-02,DC1,DC8,3
2021-01-03,DC1,DC8,3
2021-01-04,DC1,DC8,3
2021-01-05,DC1,DC8,3


In [7]:
newNoneDf.head()

,场地1,场地2,货量
2021-01-01,DC1,DC8,None
2021-01-02,DC1,DC8,None
2021-01-03,DC1,DC8,None
2021-01-04,DC1,DC8,None
2021-01-05,DC1,DC8,None


In [8]:
newExcelDf = newDf.reset_index().rename(columns={'index': '日期'}).reindex(
    columns=['场地1', '场地2', '日期', '货量'])
newExcelDf = newExcelDf.sort_values(by=['场地1', '场地2', '日期']).reset_index(drop=True)
print(newExcelDf.head())
print(newExcelDf.dtypes)
newExcelDf['日期'] = pd.to_datetime(newExcelDf['日期']).dt.date
print(newExcelDf.dtypes)
print(newExcelDf.head())
# 使用 pivot_table 转换数据
newExcelDf = pd.pivot_table(newExcelDf, values='货量', index=['场地1', '场地2'], columns='日期').reset_index()
newExcelDf.head()

   场地1  场地2         日期  货量
0  DC1  DC8 2021-01-01   3
1  DC1  DC8 2021-01-02   3
2  DC1  DC8 2021-01-03   3
3  DC1  DC8 2021-01-04   3
4  DC1  DC8 2021-01-05   3
场地1            object
场地2            object
日期     datetime64[ns]
货量              int64
dtype: object
场地1    object
场地2    object
日期     object
货量      int64
dtype: object
   场地1  场地2          日期  货量
0  DC1  DC8  2021-01-01   3
1  DC1  DC8  2021-01-02   3
2  DC1  DC8  2021-01-03   3
3  DC1  DC8  2021-01-04   3
4  DC1  DC8  2021-01-05   3


日期,场地1,场地2,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,...,2022-12-22,2022-12-23,2022-12-24,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31
0,DC1,DC8,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
1,DC10,DC12,306,328,163,193,565,964,623,284,...,320,320,320,320,320,320,320,320,320,320
2,DC10,DC13,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,DC10,DC14,2613,2941,5968,4459,7646,5106,7155,5939,...,8663,9818,12248,13255,9896,15492,8230,29021,9991,13254
4,DC10,DC17,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [9]:
newExcelDf.to_excel('均值填充-物流网络历史货量数据.xlsx', index=False)


In [10]:
newExcelDf = newNoneDf.reset_index().rename(columns={'index': '日期'}).reindex(
    columns=['场地1', '场地2', '日期', '货量'])
newExcelDf = newExcelDf.sort_values(by=['场地1', '场地2', '日期']).reset_index(drop=True)
print(newExcelDf.head())
print(newExcelDf.dtypes)
newExcelDf['日期'] = pd.to_datetime(newExcelDf['日期']).dt.date
print(newExcelDf.dtypes)
print(newExcelDf.head())
# 使用 pivot_table 转换数据
newExcelDf = pd.pivot_table(newExcelDf, values='货量', index=['场地1', '场地2'], columns='日期').reset_index()
newExcelDf.head()

   场地1  场地2         日期    货量
0  DC1  DC8 2021-01-01  None
1  DC1  DC8 2021-01-02  None
2  DC1  DC8 2021-01-03  None
3  DC1  DC8 2021-01-04  None
4  DC1  DC8 2021-01-05  None
场地1            object
场地2            object
日期     datetime64[ns]
货量             object
dtype: object
场地1    object
场地2    object
日期     object
货量     object
dtype: object
   场地1  场地2          日期    货量
0  DC1  DC8  2021-01-01  None
1  DC1  DC8  2021-01-02  None
2  DC1  DC8  2021-01-03  None
3  DC1  DC8  2021-01-04  None
4  DC1  DC8  2021-01-05  None


日期,场地1,场地2,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,...,2022-12-22,2022-12-23,2022-12-24,2022-12-25,2022-12-26,2022-12-27,2022-12-28,2022-12-29,2022-12-30,2022-12-31
0,DC1,DC8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DC10,DC12,306.0,328.0,163.0,193.0,565.0,964.0,623.0,284.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DC10,DC13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DC10,DC14,2613.0,2941.0,5968.0,4459.0,7646.0,5106.0,7155.0,5939.0,...,8663.0,9818.0,12248.0,13255.0,9896.0,15492.0,8230.0,29021.0,9991.0,13254.0
4,DC10,DC17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
newExcelDf.to_excel('None空值填充-物流网络历史货量数据.xlsx', index=False)
